## BB84


_course: quantum cryptography for beginners
<br>date: 9 november 2020
<br>author: burton rosenberg_

###  The Paper

In 1984 the first use of quantum cryptography, even quantum computing, was proposed in a small conference in Bangelor, India. Charles Bennett and Gilles Brassard, [Quantum Cryptography: Public Key Distribution and Coin Tossing](https://researcher.watson.ibm.com/researcher/files/us-bennetc/BB84highest.pdf) 1984.

It is the classic result and it combines to stages. There is a quantum algorithm, and then a classical post-processing, called *privacy amplification*. 



### The Quantum Circuit

Alice and Bob never cease to play games. They are the "it" couple for all of crypto, and seems like they now play quantum physics games.

Alice and Bob would like to agree on a random string. However, the have only a public channel over which to discuss the string, and they would like, once the string is decided, that it be known only by themselves. 

The this is sometimes called the *key agreement problem*, and it begins many cryptographic protocols. The most common use, is that the common random string is then used as an encryption key for a classical, symmetric encryption (as opposed to public-key, a.k.a. asymmetric) scheme, used in subsequent communication between Alice and Bob.

Why didn't Alice and Bob figure this out in some secret room before they parted ways? Well perhaps Alice and Bob didn't know of each other until just now, and they are separated at a distance. Perhaps they want to have a private conversation but do not want to divulge their identity, even to each other. Maybe they are using a one-time pad and have run out of pad. All these are scenarious where the key agreement problem arises.

<pre style="border:orange solid thin;padding:2em;margin:2em;">

     r_1 -----+            +----+             +----- s_1
              |            |    |             |
              V            |    |             V    
          +---+---+        |    |         +---+---+
 a_1 ---->|  Id/H |---->   |    |    ---->|  Id/H |----> b_1
          +-------+        |    |         +-------+
                           |    |
                           |    |  
     r_1 -----+            +----+             +----- s_2
              |            |    |             |
              V            |    |             V
          +---+---+        |    |         +---+---+
 a_1 ---->|  Id/H |---->   |    |    ---->|  Id/H |----> b_2
          +-------+        |    |         +-------+
                           |    |
                           |    |                  
                           +----+
                           |    |  
                           
 ...                        ...
                           
                           |    |  
     r_1 -----+            +----+             +----- s_n
              |            |    |             |
              V            |    |             V
          +---+---+        |    |         +---+---+ 
 a_1 ---->|  Id/H |---->   |    |    ---->|  Id/H |----> b_n
          +-------+        |    |         +-------+ 
                           |    |
                           |    |                  
                           +----+ 

          Alice              Eve             Bob
</pre>


Alice has her privately generate random 0/1 string, $a_1, a_2, \ldots a_n$. She generates another 0/1 string, $r_1, r_2, \ldots, r_n$. She then encodes each $a_i$ in either the $Z$ basis or the $X$ depending on the value of $r_i$,

$$
x_i = H^{r_i} |a_i\rangle.
$$

Alice sends the qubits through a communication channel, which might be tapped by eavesdropper Eve, to Bob.
Bob tossed coins to create his own random string, $s_1, s_2, \ldots, s_n$ and he attempts to decode Alice's qubits by using his $s_i$ as (not very good) guesses of $r_i$,

$$
b_i = H^{s_i} H^{r_i} |a_i\rangle = H^{s_i\oplus r_i} |a_i\rangle
$$

When $r_i = s_i$, then $b_i = a_i$, Bob recovers Alice's values. However, when $r_i \neq s_i$, then Bob's $b_i$ is a randomly chosen 0/1 bit, uncorrelated to Alice's $a_i$.

I rely on our previous learning about measurements: that a qubit measured in it's basis gives a certain outcome, but in any other basis gives an probability distribution; and when the two basis are such as $X$ and $Z$, the probability distribution erases all knowledge of the qubit's actual value.

#### Reconciliation

Then Alice and Bob, over the public channel, exchange their $r$ and $s$ strings. Now they know for which $i$ is it true that $a_i=b_i$ and they discard the remaining bits. They are left with about $n/2$ bits on which they agree and that are secret.

Why are the secret? Suppose Eve taps the channel and makes her own measurements. In order this be a traditional tap, she must forward on some qubit to Bob, after having made a measurement, either in the $Z$ basis or the $X$ basis. If she happens to make the measurement in the correct basis, her $e_i$, say, is such that $e_i=a_i$, and she also has $r_i$, so she can send on a freshly prepared qubit in the state $H^{r_i}|a_i\rangle$. However, half the time what she sends is a randomly encoded 0/1 in a randomly chosen $X/Z$ basis.

This will necessarily leave a trace. 

Once Alice and Bob exchange there random choices for basis, and discard the useless bits, they announce over the public channel half of the shared bits, $n/4$ of the total bits sent. If there has been no eavesdropping then Alice and Bob's revealed bits will agree always. However, if there has been eavesdropping, it is exponentially unlikely that they will always agree. And in fact, the error rate will be the percentage of bits that Eve has attempted to tap.

So, we now have Alice and Bob sharing two smaller strings, $\tilde{a}$ and $\tilde{b}$, respectively, of about $n/4$ (classical bits) and an estimate of $\alpha$ information leakage to Eve. That is, the full $n/4$ bits are not secret, but a theoretical limit of $\alpha\, n/4$ bits are secret.

#### Privacy amplification

What first comes to my mind, when considering Eve's eavesdropping, is her attempted detection of individual bits. This gives a certain sort of information leakage in the resulting strings &mdash; we could (in principle) point to the compromised bit, and remove it. We don't know which bit it is, but it is a localized leakage. 

Even might have made other sorts of interventions. She might have measured the parity, while retaining $n-1$ bits of information. Hence there is one bit of leakage, but it is not any particular bit. It is a slight leakage over the ensemble of all bits.

If the percentage leakage is $\alpha$ then there is an $m$ satisfying, 

$$
m \log m = \alpha \, n/2
$$

that is the entropy in bits remaining unknown to Eve. Alice and Bob wish to harvest that entropy, by agreeing, using classical means, on an $m$ bit string, $m<n/2$, of which Eve has no knowledge. 

To do this, first Alice and Bob enter an error correction protocol to reconcile $\tilde{a}$ and $\tilde{b}$, and then use a *universal hash function* to compress the resulting reconciled string down to $m$ bits.


<pre style="border:orange solid thin;padding:2em;margin:2em;">

 ALICE                         BOB
 -----                         ---
               &phi;
  random   ------------->
  r's 
  and a's 
                            random s's
                            and measure b's
              s's
         &lt;------------
          ------------>
              r's
 
glean and                   glean and 
cut a's                     cut b's
               b's
          &lt;------------
           ------------>
               a's
  
 error                      error
 correction                 correction
          &lt;--------->
          
universal                   universal
hashing                     hashing
          &lt;--------->  
<pre>

In [3]:
import qiskit
import time, math

from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.jobstatus import JOB_FINAL_STATES, JobStatus
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

qiskit.__qiskit_version__

args_g = []

# your api token from IBM, first time run.
# after that None is good

#api_token = 'abcdefghijklmnopqrstuvwxyz'
api_token = None 

def load_or_save_IBMQ_account(api_token=None):
    global args_g
    if api_token: 
    # only needs to be done once
        # then is stored in e.g. ~/.qistkit/qiskitrc
        IBMQ.save_account(api_token)
    provider = IBMQ.load_account()
# choose an alternative provider, if available
    provider = IBMQ.providers()[1]
    return provider

def list_backends(provider):
    global args_g
    backends = provider.backends()
    print('backends available:')
    for be in backends:
        st = be.status()
        if st.operational:
            print(f'\t{be.name()}, pending jobs:{st.pending_jobs}')

            
def run_quantum_circuit_on_backend(quantum_circuit,provider,backend):
    backend = provider.get_backend(backend)
    qobj = assemble(transpile(quantum_circuit, backend=backend), backend=backend)
    job = backend.run(qobj)
    return job


def wait_for_job(backend, job, wait_interval=5):
    backend = provider.get_backend(backend)
    retrieved_job = backend.retrieve_job(job.job_id())
    start_time = time.time()
    job_status = job.status()
    while job_status not in JOB_FINAL_STATES:
        print(f'Status @ {time.time() - start_time:0.0f} s: {job_status.name},'
              f' est. queue position: {job.queue_position()}')
        time.sleep(wait_interval)
        job_status = job.status()


print("listing backends ...")
provider = load_or_save_IBMQ_account(api_token)
list_backends(provider)

# choose your backend

#backend = 'ibmq_qasm_simulator'
#backend = 'ibmq_armonk'
#backend = 'ibmq_vigo'
#backend = 'ibmq_london'
#backend = 'ibmq_ourense'
#backend = 'ibmq_bogota'

# and so forth ... chose from the results given by provider.backends()

listing backends ...


ibmqfactory.load_account:WARNING:2020-11-09 16:24:57,882: Credentials are already in use. The existing account in the session will be replaced.


backends available:
	ibmq_qasm_simulator, pending jobs:0
	ibmqx2, pending jobs:872
	ibmq_16_melbourne, pending jobs:738
	ibmq_vigo, pending jobs:845
	ibmq_ourense, pending jobs:862
	ibmq_valencia, pending jobs:287
	ibmq_armonk, pending jobs:8
	ibmq_athens, pending jobs:27
	ibmq_rome, pending jobs:4
	ibmq_santiago, pending jobs:37
	ibmq_bogota, pending jobs:7
	ibmq_casablanca, pending jobs:8


In [2]:

backend = 'ibmq_qasm_simulator'
backend_obj = provider.get_backend(backend)
#backend_obj


### Builiding and runing the circuit


In [7]:
import random


class Alice:

    def __init__(self,n):
        self.n = n
        self.a = []
        self.r = []
        self.a_reconciled = []
        self.circuit = None
        self.qreg_q = None
        self.creg_c = None
        
    def random_string(self):
        return [random.randint(0,1) for i in range(self.n)]
    
    def build_circuit(self):
        self.a = self.random_string()
        self.r = self.random_string()
    
        qreg_q = QuantumRegister(self.n, 'q')
        creg_c = ClassicalRegister(self.n, 'c')
        circuit = QuantumCircuit(qreg_q, creg_c)
        for i in range(self.n):
            if self.a[i]==1:
                circuit.x(qreg_q[i])
            else:
                circuit.i(qreg_q[i])
            if self.r[i]==1:
                circuit.h(qreg_q[i])
            else:
                circuit.i(qreg_q[i])
            
        self.circuit, self.qreg_q, self.creg_c  = circuit, qreg_q, creg_c
        return (self.a, self.r)

    def describe_channel(self):
        return {'n':self.n, 'circuit':self.circuit, 
                'qreg_q':self.qreg_q, 'creg_c':self.creg_c }
    
    def reconcile(self, bob_basis):
        a_reconciled = []
        for i in range(self.n):
            if self.r[i]==bob_basis[i]:
                a_reconciled.append(self.a[i])
        self.a_reconciled = a_reconciled
        return a_reconciled
            
    def draw(self):
        print(f'alice secret: {self.a}\nbasis encoding: {self.r}')
        print('\n-------- CIRCUIT ---------')
        print(self.circuit.draw(output='text'))
        print('-------------------------\n')


class Bob:
    
    def __init__(self,channel_description):
        self.n = channel_description['n']
        self.s = []
        self.b = []
        self.b_reconciled = []
        self.circuit = channel_description['circuit']
        self.qreg_q = channel_description['qreg_q']
        self.creg_c = channel_description['creg_c']
        
    def random_string(self):
        return [random.randint(0,1) for i in range(self.n)]
    
    def build_circuit(self):
        self.s = self.random_string()

        for i in range(self.n):
            if self.s[i]==1:
                self.circuit.h(self.qreg_q[i])
            else:
                self.circuit.i(self.qreg_q[i])
            self.circuit.measure(self.qreg_q[i], self.creg_c[i])

        return (self.s)
    
    def reconcile(self, alice_basis):
        b_reconciled = []
        for i in range(self.n):
            if self.s[i]==alice_basis[i]:
                b_reconciled.append(self.b[i])
        self.b_reconciled = b_reconciled
        return b_reconciled
   
    def draw(self):
        print(f'basis encoding: {self.s}')
        print('\n-------- CIRCUIT ---------')
        print(self.circuit.draw(output='text'))
        print('-------------------------\n')
    
    def run(self):
        job = run_quantum_circuit_on_backend(self.circuit,provider,backend)
        print(f'results: waiting for results from backend {backend} ...')
        wait_for_job(backend, job)
        result = job.result()
        print(f'results: {result.get_counts()}')
        
        self.b = [[0,0] for i in range(self.n)]
        counts = result.get_counts() # a dictionary
        for count in counts:
            num = counts[count]
            for i,co in enumerate(count):
                j = self.n-i-1
                if co=='0':
                    self.b[j][0] += num
                else:
                    self.b[j][1] += num
        print(f'bobs secret: {self.b}')
        return result.get_counts()

class BB84:
    
    def __init__(self,n):
        self.n = n
        self.alice = None
        self.channel = None
        self.bob = None
        
    def build_alice(self):
        self.alice = Alice(self.n)
        self.alice.build_circuit()
        self.alice.draw()
        
    def build_channel(self):
        assert(self.alice!=None)
        self.channel = self.alice.describe_channel()

    def build_bob(self):
        assert(self.channel!=None)
        self.bob = Bob(self.channel)
        self.bob.build_circuit()
        self.bob.draw()
        
    def build_bb84(self):
        self.build_alice()
        self.build_channel()
        self.build_bob()
        
    def protocol(self):
        self.bob.run()
        self.alice.reconcile(self.bob.s)
        self.bob.reconcile(self.alice.r)
        print(f'alice: {self.alice.a_reconciled}\nbob: {self.bob.b_reconciled}')
        
bb84 = BB84(4)
bb84.build_bb84()
bb84.protocol()



alice secret: [0, 0, 1, 1]
basis encoding: [1, 1, 0, 1]

-------- CIRCUIT ---------
     ┌───┐┌───┐
q_0: ┤ I ├┤ H ├
     ├───┤├───┤
q_1: ┤ I ├┤ H ├
     ├───┤├───┤
q_2: ┤ X ├┤ I ├
     ├───┤├───┤
q_3: ┤ X ├┤ H ├
     └───┘└───┘
c: 4/══════════
               
-------------------------

basis encoding: [0, 1, 0, 1]

-------- CIRCUIT ---------
     ┌───┐┌───┐┌───┐┌─┐         
q_0: ┤ I ├┤ H ├┤ I ├┤M├─────────
     ├───┤├───┤├───┤└╥┘┌─┐      
q_1: ┤ I ├┤ H ├┤ H ├─╫─┤M├──────
     ├───┤├───┤├───┤ ║ └╥┘┌─┐   
q_2: ┤ X ├┤ I ├┤ I ├─╫──╫─┤M├───
     ├───┤├───┤├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ X ├┤ H ├┤ H ├─╫──╫──╫─┤M├
     └───┘└───┘└───┘ ║  ║  ║ └╥┘
c: 4/════════════════╩══╩══╩══╩═
                     0  1  2  3 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'1100': 531, '1101': 493}
bobs secret: [[531, 493], [1024, 0], [0, 1024], [0, 1024]]
alice: [0, 1, 1]
bob: [[1024, 0], [0, 1024], [0, 1024